# Downloading and pre-processing marjanovic et. al 2020 data

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
from urllib.request import urlretrieve

import numpy as np
import pandas as pd
import scanpy as sc

from gepdynamics import _utils


print(os.getcwd())
os.chdir('/cs/labs/mornitzan/yotamcon/gep-dynamics')


/cs/labs/mornitzan/yotamcon/gep-dynamics/notebooks


In [2]:
results_dir = _utils.set_dir('results')
orig_adata_path = results_dir.joinpath('marjanovic_mmLungPlate.h5ad')

if not orig_adata_path.exists():  # create the original adata if it doesn't exist
    # directories for file download:
    data_dir = _utils.set_dir('data')
    GSE_dir = _utils.set_dir(data_dir.joinpath('GSE154989'))
    
    # GEO server prefix for mmLungPlate SubSeries GSE154989
    ftp_address = 'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE154nnn/GSE154989/suppl/'

    #filenames
    f_rawCount = GSE_dir.joinpath('GSE154989_mmLungPlate_fQC_dSp_rawCount.h5')
    f_geneTable = GSE_dir.joinpath('GSE154989_mmLungPlate_fQC_geneTable.csv.gz')
    f_smpTable = GSE_dir.joinpath('GSE154989_mmLungPlate_fQC_smpTable.csv.gz')
    f_smp_annot = GSE_dir.joinpath('GSE154989_mmLungPlate_fQC_dZ_annot_smpTable.csv.gz')

    ftp_address = 'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE154nnn/GSE154989/suppl/'

    # downloading if needed:
    if not f_rawCount.exists():
        urlretrieve(ftp_address + f_rawCount.name, f_rawCount)
    
    if not f_geneTable.exists():
        urlretrieve(ftp_address + f_geneTable.name, f_geneTable)
    
    if not f_smpTable.exists():
        urlretrieve(ftp_address + f_smpTable.name, f_smpTable)
    
    if not f_smp_annot.exists():
        urlretrieve(ftp_address + f_smp_annot.name, f_smp_annot)
    
    # reading the files
    sparse_counts = _utils.read_matlab_h5_sparse(f_rawCount)
    
    gene_ids = pd.read_csv(f_geneTable, index_col=0)
    smp_ids = pd.read_csv(f_smpTable, index_col=0)
    smp_annotation = pd.read_csv(f_smp_annot, index_col=0)
    
    # constructing the adata
    adata = sc.AnnData(X=sparse_counts, dtype=np.float64, var=gene_ids, obs=smp_ids)
    
    adata.obs['clusterK12'] = smp_annotation.clusterK12
    
    adata.obsm['X_tsne'] = smp_annotation[['tSNE_1', 'tSNE_2']].values
    adata.obsm['X_phate'] = smp_annotation[['phate_1', 'phate_2']].values
    adata.write(orig_adata_path)
else:
    adata = sc.read(orig_adata_path)

adata

AnnData object with n_obs × n_vars = 3891 × 52638
    obs: 'plateID', 'mouseID', 'timesimple', 'typeID', 'clusterK12'
    var: 'ensgID', 'geneSymbol', 'length'
    obsm: 'X_phate', 'X_tsne'

In [4]:
adata.obs_keys()

['plateID', 'mouseID', 'timesimple', 'typeID', 'clusterK12']

In [9]:
adata.obs.timesimple.value_counts()

08_KP_30w_ND         1554
07_KP_20w_ND          538
05_K_30w_ND           505
06_KP_12w_ND          491
04_K_12w_ND           455
01_T_early_ND         206
02_KorKP_early_ND     142
Name: timesimple, dtype: int64